# Making A Neural Network From Scratch:

**Goal**: To implement a neural network using only numPy that has a one hidden layer and one output layer. We can then train our neural network on the MNIST data-set, and test it to see our accuracy. 

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/chinese-mnist-digit-recognizer/chineseMNIST.csv
/kaggle/input/mnist-digit-recognizer/train.csv


I'm going to first import the mnist digits in csv form, every row represents an image and every column, a pixel, with the first column being the label of the number.

In [2]:
df = pd.read_csv("/kaggle/input/mnist-digit-recognizer/train.csv")
df.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
df.shape # rows show how many pictures there are in the csv 

(42000, 785)

We are going to convert this DataFrame into a np.array, and use test_train_split() by sklearn. Then we want to take the **tranpose** of each array so that each array is actually an individual picture (which will be one of the 784 nodes).

In [4]:
x_train, x_test, y_train, y_test = train_test_split(np.array(df.iloc[:, 1:]), np.array(df[['label']]), test_size = .4)
x_train, x_test, y_train, y_test = x_train.T, x_test.T, y_train.T, y_test.T 
pixels = df.shape[1] - 1
display(x_train[:, 0].shape, len(x_test[0]), y_train, y_test) # check configs
# n_xtrain (rows of pixels) = 784, m_xtrain (columns of items) = train/split

(784,)

16800

array([[0, 0, 9, ..., 6, 1, 8]])

array([[0, 3, 2, ..., 1, 3, 3]])

**Note**: Basically we want to make the train set an nxm matrix so we can left multiply it by a weight matrix to move from Rm -> R10 (10 is an arbitrary number for the first layer but makes computation easier). This will help to compute a linear combination with the weights and nodes 

# Initialization and Propagation: 
1. weights
2. biases 
3. arrays for each layer:
    * input layer
    * first reLU layer (a function applied to each node)
    * second output layer (which will also have a softmax to find a probability distribution of numbers)
4. Make forward and backwards propagation. 

We are also going to start with random weights and biases. Below is a diagram in LaTeX explaining the matrix multiplcation, where matrix $B$ is our input layer tranposed multiplied by our $A$ matrix, and the biases being added as matrix $C$. The result will be a $10 * m$ matrix, where $m$ is the number of training pictures.





$$\begin{bmatrix}
a_{11} & a_{12} & \cdots & a_{1,784} \\
a_{21} & a_{22} & \cdots & a_{2,784} \\
\vdots & \vdots & \ddots & \vdots \\
a_{10,1} & a_{10,2} & \cdots & a_{10,784}
\end{bmatrix}
\times
\begin{bmatrix}
b_{11} & b_{12} & \cdots & b_{1m} \\
b_{21} & b_{22} & \cdots & b_{2m} \\
\vdots & \vdots & \ddots & \vdots \\
b_{784,1} & b_{784,2} & \cdots & b_{784,m}
\end{bmatrix}
+
\begin{bmatrix}
c_{11} & c_{12} & \cdots & c_{1m} \\
c_{21} & c_{22} & \cdots & c_{2m} \\
\vdots & \vdots & \ddots & \vdots \\
c_{10,1} & c_{10,2} & \cdots & c_{10,m}
\end{bmatrix}
$$


Here are the softmax function and ReLU function definitions:

$
\text{softmax}(z_i) = \frac{e^{z_i}}{\sum_{j=1}^n e^{z_j}}
$

where $z_i$ is the i-th element of the input vector z. The softmax function will apply this to each $z_i$.

The softmax function is applied independently to each column of $z$. For a single column (logits for one image), softmax computes that equation.

$
\text{ReLU}(x) = \max(0, x)
$


In [5]:
def init_params(): 
    # populate a 10 x 784 matrix with random numbers, W1 is weight for first layer
    W1 = np.random.rand(10, pixels) 
    # continue with biases, which should be a 10x1 vector, which we will broadcast
    # 10x1 -> 10xm so we can add the dot product + b
    b1 = np.random.rand(10, 1) 
    # second layer weights and biases
    W2 = np.random.rand(10, 10) 
    b2 = np.random.rand(10, 1)
    return W1, b1, W2, b2
def reLU(Z): # return the same sized matrix, applied reLU
    for row in Z: 
        for i in range(row): 
            if row[i] < 0: 
                row[i] = 0;
    return Z
    # better solution use np.maximum(0, Z)
def softMax(Z): # input: 10 x m matrix
    return np.exp(Z) / np.sum(np.exp(Z)) 

    # 1. np.exp(Z) applied to every element in Z 
    # 2. np.sum(np.exp(Z)) sums all the values of Z from exp, scalar
    # 3. All elements are normalized, result = 10 x m, for each training result
def forward_propagation(W1, b1, W2, b2, X): # X is the initial input matrix
    # Z1 will be the resulting matrix from matrix multiplication
    Z1 = W1.dot(X) + b1 
    # apply reLU to change from linear combination -> non-linear function
    A1 = ReLu(Z1)
    Z2 = W2.dot(A1) + b2 # 10 x m
    A2 = SoftMax(Z2) # predictions
    # return values to update in backpropgation
    return Z1, A1, Z2, A2

## Back Propagation and Updating Weights
We will now be writing the backpropagation part: https://www.youtube.com/watch?v=Ilg3gGewQ5U&list=PLZHQObOWTQDNU6R1_67000Dx_ZCJB-3pi&index=3

1. We need to write one hot (which is the vector equivalent of what the "actual" value is)
2. write down these equations, which are partial derivatives. z is the error (or called cost) of each of the columns (not to be confused with the previous z), dw is the change in weights with respect to the cost da is the change in activation (which is actually just activation(dz, the change in weighted sum) and db is the change in bias. Here are the equations for singular nodes: 

$\frac{\partial C_0}{\partial w^{(L)}} = \frac{\partial z^{(L)}}{\partial w^{(L)}} \frac{\partial a^{(L)}}{\partial z^{(L)}} \frac{\partial C_0}{\partial a^{(L)}}$

$z^{(L)} = w^{(L)} a^{(L-1)} + b^{(L)}$ <br>
$a^{(L)} = \sigma(z^{(L)})$ <br>
$C^{(0)} = a^{(L)} - y$


3. Using these equations, we can compute a gradient, which for each layer is a vector that tells us the mangitutde of the effect (cost) of each partial derivative. 

In [6]:
def one_hot(Y): # input: the example labels, output: all the one hot Y's
    one_hot_Y = np.zeros((Y.size, Y.max() + 1)) # (#examples, 0-9 *10), we will take transpose later
    for row in one_hot_Y: 
        for label in y_train: 
           one_hot_Y[row, label] = 1 # labels are 0 based
    # even better one_hot is one_hot_Y[np.arrange(Y.size), Y] = 1 muah
    return one_hot_Y.T
def back_propagation(Z1, A1, Z2, A2, W2, Y): 
    m = Y.size # number examples
    
    dC2 = A2 - one_hot(Y) # dZ2 = predicted - actual, this is the cost
    # above is a 10 x m cost matrix per example
    
    dB2 = 1/m * np.sum(dC2, 2) 
    dW2 = 1/m * dC2.dot(A1.T) # partial d of the second costs to weights
    dC1 = W2.T.dot(dC2) * deriative_ReLU
# the A2.T activation transpose serves to "go backwards" in a sense

Now, we'll go the opposite way and calculate how to nudge our parameters to carry out gradient descent.

Mathematically, what we're actually computing is the derivative of the loss function with respect to each weight and bias parameter. For a softmax classifier, we'll use a cross-entropy loss function:

$$J(\hat{y}, y) = -\sum_{i=0}^{c} y_i \log(\hat{y}_i)$$

Here, $$\hat{y}$$ is our prediction vector. It might look like this:

$$\begin{bmatrix} 0.01 \ 0.02 \ 0.05 \ 0.02 \ 0.80 \ 0.01 \ 0.01 \ 0.00 \ 0.01 \ 0.07 \ \end{bmatrix}$$

$$y$$ is the one-hot encoding of the correct label for the training example. If the label for a training example is 4* , for example, the one-hot encoding of $$y$$ would look like this:

$$\begin{bmatrix} 0 \ 0 \ 0 \ 0 \ 1 \ 0 \ 0 \ 0 \ 0 \ 0 \ \end{bmatrix}$$

Notice that in our sum $$\sum_{i=0}^{c} y_i \log(\hat{y}_i)$$, $$y_i = 0$$ for all $$i$$ except the correct label. The loss for a given example, then, is just the log of the probability given for the correct prediction. In our example above, $$J(\hat{y}, y) = -\log(y_4) = -\log(0.80) \approx 0.097$$. Notice that, the closer the prediction probability is to 1, the closer the loss is to 0. As the probability approaches 0, the loss approaches $$+\infty$$. By minimizing the cost function, we improve the accuracy of our model. We do so by substracting the derivative of the loss function with respect to each parameter from that parameter over many rounds of graident descent:

$$W^{[1]} := W^{[1]} - \alpha \frac{\delta J}{\delta W^{[1]}} \ b^{[1]} := b^{[1]} - \alpha \frac{\delta J}{\delta b^{[1]}} \ W^{[2]} := W^{[2]} - \alpha \frac{\delta J}{\delta W^{[2]}} \ b^{[2]} := b^{[2]} - \alpha \frac{\delta J}{\delta b^{[2]}} \ $$

Our objective in backprop is to find $$\frac{\delta J}{\delta W^{[1]}},\frac{\delta J}{\delta b^{[1]}},\frac{\delta J}{\delta W^{[2]}},$$ and $$\frac{\delta J}{\delta b^{[2]}}$$. For concision, we'll write these values as $$dW^{[1]}, db^{[1]}, dW^{[2]},$$and $$db^{[2]}$$. We'll find these values by stepping backwards through our network, starting by calculating $$\frac{\delta J}{\delta A^{[2]}}$$, or $$dA^{[2]}$$. Turns out that this derivative is simply:

$$dA^{[2]} = Y - A^{[2]}$$

If you know calculus, you can take the derivative of the loss function and confirm this for yourself. (Hint: $$\hat{y} = A^{[2]}$$)

From $$dA^{[2]}$$, we can calculate $$dW^{[2]}$$ and $$db^{[1]}$$:

$$dW^{[2]} = \frac{1}{m} dZ^{[2]} A^{[1]T} \ dB^{[2]} = \frac{1}{m} \Sigma {dZ^{[2]}}$$

Then, to calculate $$dW^{[1]}$$ and $$db^{[1]}$$, we'll first find $$dZ^{[1]}$$:

$$dZ^{[1]} = W^{[2]T} dZ^{[2]} .* g^{[1]\prime} (Z^{[1]})$$

I won't explain all the details of the math, but you can get some intuitive hints at what's going on here by just looking at the variables. We're applying $$W^{[2]T}$$ to $$dZ^{[2]}$$, akin to applying the weights between layers 1 and 2 in reverse. Then, we perform an element-wise multiplication with the derivative of the activation function, akin to "undoing" it to get the correct error values.

Since our activation function is ReLU, our derivative is actually pretty simple. Let's revisit our graph: